In [11]:
import config as c
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

# Descriptive Analysis
- demographic data
- fitts' law
    - effective id
    - task completion time (overall & per id)
    - error rate (overall & per id)
    - throughput (overall & per id)
    - regression of error rate
    - regression of task completion time
- nasa-tlx

## Analysis of Demographic Data

In [12]:
demographic_df = pd.DataFrame()
for id in range(c.NUM_PARTICIPANTS):
    demographic_raw = pd.read_csv(
        c.FOLDER_PATH
        + str(id + 1)
        + f"/demographic_participant_{id + 1}.csv"
    )
    demographic_df = pd.concat([demographic_df, demographic_raw])

demographic_df.head()

,participant_id,agreed,age,gender,hand,input_device,mouse
0,1,True,23,male,right,Mouse,often


In [13]:
mean_age = np.mean(demographic_df["age"])
std_age = np.std(demographic_df["age"])
print("Average age: ", mean_age, " (Std: ", std_age, ")")
print(
    "Gender: \n",
    demographic_df["gender"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "value", 0: "count"}),
)
print(
    "Handedness: \n",
    demographic_df["hand"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "value", 0: "count"}),
)
print(
    "Most used input device: \n",
    demographic_df["input_device"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "value", 0: "count"}),
)
print(
    "Frequency of mouse input: \n",
    demographic_df["mouse"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "value", 0: "count"}),
)

Average age:  23.0  (Std:  0.0 )
Gender: 
   gender  count
0   male      1
Handedness: 
     hand  count
0  right      1
Most used input device: 
   input_device  count
0        Mouse      1
Frequency of mouse input: 
    mouse  count
0  often      1


## Analyis of Fitts' Law 

In [14]:
clicks_df = pd.DataFrame()
for id in range(c.NUM_PARTICIPANTS):
    for trial in range(c.NUM_TRIALS):
        clicks_raw = pd.read_csv(
            c.FOLDER_PATH
            + str(id + 1)
            + f"/clicks_participant_{id + 1}_trial_{trial}.csv"
        )
        clicks_df = pd.concat([clicks_df, clicks_raw])
    
clicks_df.head()

,id,timestamp_ms,participant_id,trial,level_of_latency,target_number,target_width,target_amplitude,target_x,target_y,cursor_x,cursor_y,success,completion_time
0,0,1710166191603,1,0,0,0,30,450,959,103,957,107,1,1.787344
1,1,1710166193111,1,0,0,1,30,450,1113,975,1117,968,1,1.508388
2,2,1710166194311,1,0,0,2,30,450,670,208,672,211,1,1.199971
3,3,1710166196228,1,0,0,3,30,450,1349,778,1353,787,1,1.916836
4,4,1710166197495,1,0,0,4,30,450,516,474,519,467,1,1.267135


In [15]:
# drop irrelevant columns
clicks_df = clicks_df.drop(
    ["timestamp_ms", "level_of_latency", "target_number"], axis=1
)
# drop rows of first click in round (no fitts law task)
clicks_df = clicks_df.drop(clicks_df.loc[clicks_df["id"] % 9 == 0].index, axis=0)
clicks_df.reset_index(inplace=True, drop=True)
clicks_df.head()

,id,participant_id,trial,target_width,target_amplitude,target_x,target_y,cursor_x,cursor_y,success,completion_time
0,1,1,0,30,450,1113,975,1117,968,1,1.508388
1,2,1,0,30,450,670,208,672,211,1,1.199971
2,3,1,0,30,450,1349,778,1353,787,1,1.916836
3,4,1,0,30,450,516,474,519,467,1,1.267135
4,5,1,0,30,450,1403,474,1409,482,1,1.045496


### Overall Fitts' Law Analysis
- Calculation of id
- Calculation of effective id
- Mean error rate
- Mean completion time
- Mean throughput

In [16]:
# calculation of id
clicks_df["ID"] = np.log2(clicks_df["target_amplitude"] / clicks_df["target_width"] + 1)

In [19]:
# calculation of effective id
num_ids = len(clicks_df["ID"].unique())

clicks_df["distance_from_target"] = np.sqrt(
    (clicks_df["target_x"] - clicks_df["cursor_x"])**2 +
    (clicks_df["target_y"] - clicks_df["cursor_y"])**2
)

effective_id_df = pd.DataFrame({"ID": clicks_df["ID"].unique(), 
                                "sigma": np.zeros(num_ids),
                                "amplitude": np.zeros(num_ids),
                                "width": np.zeros(num_ids)})
for _, entry in clicks_df.iterrows():
    for _, row in effective_id_df.iterrows():
        if entry["ID"] == row["ID"]:
            row["amplitude"] = entry["target_amplitude"]
            row["width"] = entry["target_width"]
            row["sigma"] += entry["distance_from_target"]

effective_id_df["sigma"] /= len(clicks_df)/9
effective_id_df["We"] = 4.133 * effective_id_df["sigma"]
effective_id_df["IDe"] = np.log2(
    effective_id_df["amplitude"] / effective_id_df["We"] + 1
)
effective_id_df.sort_values(by=["ID"])

,ID,sigma,amplitude,width,We,IDe
5,1.000000,21.250124,200.0,200.0,87.826763,1.712468
3,1.392317,39.384357,325.0,200.0,162.775547,1.583333
8,1.415037,22.420815,200.0,120.0,92.665228,1.659151
6,1.700440,37.300828,450.0,200.0,154.164321,1.970472
2,1.890771,24.311747,325.0,120.0,100.480452,2.082178
7,2.247928,24.692711,450.0,120.0,102.054975,2.435465
4,2.938599,8.416105,200.0,30.0,34.783763,2.754847
1,3.564785,8.197249,325.0,30.0,33.879229,3.405025
0,4.000000,8.564733,450.0,30.0,35.398040,3.777427


In [ ]:
def calculate_mean_std(arr):
    return round(np.mean(arr), 2), round(np.std(arr), 2)

In [21]:
# calculation of mean and standard deviation of error rate and completion time
print("Mean error rate: ",  calculate_mean_std(1 - clicks_df["success"]))
print("Mean completion time: ",  calculate_mean_std(clicks_df["completion_time"]))

Mean error rate:  (0.03, 0.17)
Mean completion time:  (0.56, 0.19)


In [22]:
# calculation of throughput
throughput_df = pd.DataFrame({"IDe": np.zeros(clicks_df.shape[0]),
                              "completion_time": clicks_df["completion_time"],
                              "error_rate": 1 - clicks_df["success"]})

for index, entry in clicks_df.iterrows():
    for _, row in effective_id_df.iterrows():
        if entry["ID"] == row["ID"]:
            throughput_df.loc[index,"IDe"] = row["IDe"]

throughput_df["throughput"] = throughput_df["IDe"] / throughput_df["completion_time"]
print("Mean throughput: ",  calculate_mean_std(throughput_df["throughput"]))

Mean throughput:  (4.35, 0.72)


In [23]:
# calculation of throughput, error rate, completion time per id
ides = throughput_df["IDe"].unique()
throughput_df_per_id = pd.DataFrame({"IDe": ides,
                                     "mean_throughput": np.zeros(len(ides)),
                                     "std_throughput": np.zeros(len(ides)),
                                     "mean_completion_time": np.zeros(len(ides)),
                                     "std_completion_time": np.zeros(len(ides)),
                                     "mean_error_rate": np.zeros(len(ides)),
                                     "std_error_rate": np.zeros(len(ides))})

for _, entry in throughput_df_per_id.iterrows():
    id = entry["IDe"]
    queried_df = throughput_df.query(f"IDe=={id}")
    throughput = queried_df["throughput"]
    completion_time = queried_df["completion_time"]
    error_rate = queried_df["error_rate"]
    entry["mean_throughput"], entry["std_throughput"] = calculate_mean_std(throughput)
    entry["mean_completion_time"], entry["std_completion_time"] = calculate_mean_std(completion_time)
    entry["mean_error_rate"], entry["std_error_rate"] = calculate_mean_std(error_rate)

throughput_df_per_id["IDe"] = round(throughput_df_per_id["IDe"], 2)
throughput_df_per_id = throughput_df_per_id.sort_values(by=["IDe"])
throughput_df_per_id

,IDe,mean_throughput,std_throughput,mean_completion_time,std_completion_time,mean_error_rate,std_error_rate
3,1.58,4.04,0.60,0.40,0.06,0.00,0.00
8,1.66,4.45,0.70,0.38,0.07,0.00,0.00
5,1.71,4.15,0.74,0.43,0.10,0.00,0.00
6,1.97,3.99,0.65,0.51,0.11,0.02,0.14
2,2.08,4.47,0.63,0.48,0.09,0.01,0.08
7,2.44,4.31,0.59,0.58,0.09,0.05,0.22
4,2.75,4.46,0.73,0.64,0.11,0.04,0.19
1,3.41,4.64,0.73,0.75,0.13,0.06,0.24
0,3.78,4.62,0.79,0.85,0.18,0.10,0.30
